<a href="https://colab.research.google.com/github/Carmen-N/Python-Recession-Case--Study/blob/master/An%C3%A1lisis_de_la_recesi%C3%B3n_con_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La recesión se calcula y analiza en función del crecimiento del PIB, el crecimiento de la tasa de desempleo y el crecimiento de la tasa de gasto de los consumidores. Pero la forma más común de medir la recesión es analizando los datos mensuales de crecimiento del PIB.

Ahora vamos a empezar esta tarea de análisis de la Recesión importando las librerías Python necesarias y el conjunto de datos:

In [7]:
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio 
pio.templates.default = "plotly_white"
data = pd.read_csv('/UK_monthly_gdp.csv')
print(data.head())

  Time Period  GDP Growth
0    /01/2020         0.3
1    /02/2020        -0.5
2    /03/2020        -7.0
3    /04/2020       -20.9
4    /05/2020         3.2


Veamos el aumento del PIB a lo largo del tiempo:

In [9]:
fig = go.Figure(data=go.Heatmap(
                   z=[data['GDP Growth']],
                   x=data.index,
                   y=['GDP Growth'],
                   colorscale='Viridis'))

fig.update_layout(title='Crecimiento del PIB en el tiempo',
                  xaxis_title='Periodo de tiempo',
                  yaxis_title='')

fig.show()

Como una recesión significa la disminución de la circulación de dinero durante dos trimestres consecutivos, convertiré nuestros datos mensuales en trimestrales para analizar la recesión:

In [10]:
# Convertir los datos mensuales en trimestrales utilizando el método de remuestreo

data['Time Period'] = pd.to_datetime(data['Time Period'], format='/%m/%Y')
data.set_index('Time Period', inplace=True)
quarterly_data = data.resample('Q').mean()
print(quarterly_data.head())

             GDP Growth
Time Period            
2020-03-31    -2.400000
2020-06-30    -2.900000
2020-09-30     3.500000
2020-12-31     0.200000
2021-03-31     0.033333


Ahora podemos calcular y analizar la recesión basándonos en el crecimiento trimestral del PIB:

In [13]:
# Calcular la recesión a partir del crecimiento trimestral del PIB
quarterly_data['Recession'] = ((quarterly_data['GDP Growth'] < 0) & (quarterly_data['GDP Growth'].shift(1) < 0))

In [14]:
# Rellenar los valores que faltan con Falso (ya que el primer trimestre no puede estar en recesión)
quarterly_data['Recession'].fillna(False, inplace=True)

In [17]:
# Representar gráficamente los datos de crecimiento y recesión del PIB
fig = go.Figure()
fig.add_trace(go.Scatter(x=quarterly_data.index, 
                         y=quarterly_data['GDP Growth'], 
                         name='GDP Growth', 
                         line=dict(color='green', width=2)))
fig.add_trace(go.Scatter(x=quarterly_data[quarterly_data['Recession']].index, 
                         y=quarterly_data[quarterly_data['Recession']]['GDP Growth'], 
                         name='Recession', line=dict(color='red', width=2)))

fig.update_layout(title='Crecimiento del PIB y recesión a lo largo del tiempo (datos trimestrales)',
                  xaxis_title='Periodo de tiempo',
                  yaxis_title='Aumento del PIB')

fig.show()

La línea roja muestra los periodos de crecimiento negativo del PIB (considerados recesiones), y la línea verde muestra la tendencia general del crecimiento del PIB a lo largo del tiempo.

Analicemos ahora la gravedad de la recesión. La severidad de una recesión se refiere al grado de contracción de la economía durante una recesión. Una recesión severa implica un descenso más profundo y prolongado de la actividad económica, que tiene efectos negativos sobre el empleo, los ingresos y otros indicadores económicos. He aquí cómo analizar la gravedad de la recesión:

In [19]:
quarterly_data['Recession Start'] = quarterly_data['Recession'].ne(quarterly_data['Recession'].shift()).cumsum()
recession_periods = quarterly_data.groupby('Recession Start')
recession_duration = recession_periods.size()
recession_severity = recession_periods['GDP Growth'].sum()
fig = go.Figure()
fig.add_trace(go.Bar(x=recession_duration.index, y=recession_duration,
                     name='Períodos de recesión'))
fig.add_trace(go.Bar(x=recession_severity.index, y=recession_severity,
                     name='Intensidad de la recesión'))

fig.update_layout(title='Duración e intensidad de la recesión',
                  xaxis_title='Períodos de recesión',
                  yaxis_title='Duración/ Severidad')

fig.show()

Resumen
Una recesión es una situación económica que llega cuando la circulación de dinero en la economía es baja durante dos trimestres consecutivos. La recesión se calcula y analiza en función del crecimiento del PIB, el crecimiento de la tasa de desempleo y el crecimiento de la tasa de gasto de los consumidores. 